In [ ]:
# |default_exp summarization
# |default_cls_lvl 3

In [ ]:
# |hide
%reload_ext autoreload
%autoreload 2

# summarization

Training, saving, and tuning code for building summariztion model(s) that can predict both headlines and short summaries of topics given the text associated to a topic segment

In [ ]:
# |export
from __future__ import annotations

import os, warnings

from fastcore.all import *
from fastai.imports import *
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers.utils import logging as hf_logging

from course_copilot import utils, training

In [ ]:
# | hide
import pdb

from fastcore.test import *
import nbdev

from blurr.utils import print_versions

In [ ]:
# |export
# silence all the HF warnings
warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

In [ ]:
# | echo: false
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")

What we're running with at the time this documentation was generated:
torch: 1.12.1+cu102
fastai: 2.7.9
transformers: 4.22.0


In [ ]:
# |hide
# |cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")

Using GPU #1: GeForce GTX 1080 Ti


## Setup

## Export -

In [ ]:
# | hide
nbdev.nbdev_export()